In [1]:
import os

# --- Define Paths ---
install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
gromacs_env_script = os.path.join(install_dir, "gromacs_env.sh")
gmx_executable_path = os.path.join(install_dir, "bin/gmx")
conda_env_name = "gmxmmpbsa"

# 1. Mount Google Drive
print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# 2. CRITICAL FIX: Re-apply execute permission for GROMACS (Solves 'Permission denied')
print(f"Restoring execute permission for GROMACS: {gmx_executable_path}")
!chmod +x {gmx_executable_path}

# 3. Source GROMACS environment variables (for the current shell/Python process)
print(f"Sourcing GROMACS environment script: {gromacs_env_script}")
!source {gromacs_env_script}

# 4. Re-initialize Condacolab (Required for gmxMMPBSA)
print("\nInstalling condacolab. This will RESTART the runtime.")
print("After the restart, run the next cell (Step 2).")
!pip install -q condacolab
import condacolab
condacolab.install()

Mounting Google Drive...
Mounted at /content/drive
Restoring execute permission for GROMACS: /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Sourcing GROMACS environment script: /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh

Installing condacolab. This will RESTART the runtime.
After the restart, run the next cell (Step 2).
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [2]:
import os
import subprocess

conda_env_name = "gmxmmpbsa"

# Ensure the Conda shell is initialized for this Python session (redundant but safe)
!source /usr/local/etc/profile.d/conda.sh

# --- Creating 'gmxmmpbsa' Conda Environment and Installing Tools ---
print(f"\n--- Creating '{conda_env_name}' Conda Environment and Installing Tools ---")

# 1. Create Conda Environment
!conda create -n {conda_env_name} python=3.9 -y

# 2. Install AmberTools & Compilers
print("\nInstalling AmberTools, Compilers, and MPI...")
!conda run -n {conda_env_name} conda install -c conda-forge ambertools compilers mpi4py -y

# 3. Install All Python Analysis Tools
print("\nInstalling gmx_MMPBSA and other Python tools...")
!conda run -n {conda_env_name} pip install gmx_MMPBSA acpype MDAnalysis prolif pandas matplotlib seaborn

print(f"\n--- ✅ Installation of Analysis Tools Finished in Current Runtime ---")


--- Creating 'gmxmmpbsa' Conda Environment and Installing Tools ---
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/gmxmmpbsa

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    ld_impl_linux-64-2.44      |       ha97dd6f_2         730 KB  conda-forge
    libexpat-2.7.1             |       hecca717_0          73 KB  conda-forge
    libffi-3.5.2               |       h9ec8514_0          56 KB  conda-forge
    libgcc-1

In [3]:
print("--- Installing ImageMagick and DFT Dependencies ---")
!apt-get update -y
!apt-get install -y libxc-dev liblapack-dev libblas-dev imagemagick

# Verify ImageMagick (convert)
print("\n--- 4. ImageMagick Verification ---")
!convert -version

print("\n--- ✅ All tools are now 100% ready for use! ---")

--- Installing ImageMagick and DFT Dependencies ---
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 https://developer.download.nvidia

In [ ]:
# --- Verification (Run after the installation cell above) ---
import os

# Define paths
install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
gromacs_env_script = os.path.join(install_dir, "gromacs_env.sh")
gmx_executable_path = os.path.join(install_dir, "bin/gmx")
conda_env_name = "gmxmmpbsa"

# CRITICAL FIX 1: GROMACS Environment Setup
# Re-fix and Re-source the GROMACS path
!chmod +x {gmx_executable_path}

# CRITICAL FIX 2: CONDA Environment Re-Initialization and Verification
print("\n--- Running final verification via combined bash script... ---")

# Execute the combined verification script (using the correct f-string syntax)
script_content = f"""#!/bin/bash
# 1. Manually source the Conda initialization script
source /usr/local/etc/profile.d/conda.sh

# 2. Source the GROMACS environment script
source {gromacs_env_script}

echo ""
echo "--- 1. GROMACS Verification ---"
gmx -version

echo ""
echo "--- 2. gmxMMPBSA Verification ---"
conda run -n {conda_env_name} gmx_MMPBSA -v

echo ""
echo "--- 3. AmberTools (antechamber) Verification ---"
conda run -n {conda_env_name} antechamber -h

echo ""
echo "--- 4. ImageMagick Verification ---"
convert -version
"""

# Write the script content to the file
with open("final_verification_check.sh", "w") as f:
    f.write(script_content)

# Execute the combined verification script
!bash final_verification_check.sh


--- Running final verification via combined bash script... ---

--- 1. GROMACS Verification ---
                         :-) GROMACS - gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content
Command line:
  gmx -version

GROMACS version:     2025.3
Precision:           mixed
Memory model:        64 bit
MPI library:         thread_mpi
OpenMP support:      enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:         CUDA
NBNxM GPU setup:     super-cluster 2x2x2 / cluster 8 (cluster-pair splitting on)
SIMD instructions:   AVX2_256
CPU FFT library:     fftw-3.3.10-sse2-avx-avx2-avx2_128
GPU FFT library:     cuFFT
Multi-GPU FFT:       none
RDTSCP usage:        enabled
TNG support:         enabled
Hwloc support:       disabled
Tracing support:     disabled
C compiler:          /usr/bin/cc GNU 11.4.0
C compiler flags:    -fexcess-precision=fast -funroll-all-loops -mavx2 -mfma -Wno-m

In [2]:
%%bash

# Define paths (optional, but clean)
GROMACS_INSTALL_DIR="/content/drive/MyDrive/Colab_Software/GROMACS"
ENV_SCRIPT="${GROMACS_INSTALL_DIR}/gromacs_env.sh"

# Source environment for this bash session
source ${ENV_SCRIPT}

# Now gmx should be available
gmx --version

                         :-) GROMACS - gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content
Command line:
  gmx --version

GROMACS version:     2025.3
Precision:           mixed
Memory model:        64 bit
MPI library:         thread_mpi
OpenMP support:      enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:         CUDA
NBNxM GPU setup:     super-cluster 2x2x2 / cluster 8 (cluster-pair splitting on)
SIMD instructions:   AVX2_256
CPU FFT library:     fftw-3.3.10-sse2-avx-avx2-avx2_128
GPU FFT library:     cuFFT
Multi-GPU FFT:       none
RDTSCP usage:        enabled
TNG support:         enabled
Hwloc support:       disabled
Tracing support:     disabled
C compiler:          /usr/bin/cc GNU 11.4.0
C compiler flags:    -fexcess-precision=fast -funroll-all-loops -mavx2 -mfma -Wno-missing-field-initializers -O3 -DNDEBUG
C++ compiler:        /usr/bin/c++ GNU 11.4.0
C++ compiler

In [3]:
# Define the target directory path
target_dir = "/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF"

# Create the directory if it doesn't exist (ensures your path is valid)
import os
os.makedirs(target_dir, exist_ok=True)

# Change the current working directory using the %cd magic command
print(f"Changing directory to: {target_dir}")
%cd {target_dir}

# Verify the change
!pwd
!ls -F # List contents to confirm you are in the right place

Changing directory to: /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF
/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF
/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF
Control/		     gmx_MMPBSA.log  lupeol/
final_verification_check.sh  hedragenin/     maslinic_acid/


In [4]:
# Check GPU CUDA core information
!nvidia-smi -q


==============NVSMI LOG==============

Timestamp                                 : Wed Oct 22 04:56:03 2025
Driver Version                            : 550.54.15
CUDA Version                              : 12.4

Attached GPUs                             : 1
GPU 00000000:00:03.0
    Product Name                          : NVIDIA L4
    Product Brand                         : NVIDIA
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number          

In [5]:
# Check CPU information
import multiprocessing

print(f"Number of CPU cores available: {multiprocessing.cpu_count()}")

Number of CPU cores available: 12


In [6]:
# Check GPU information (if available)
import tensorflow as tf

gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print("GPU is available.")
    print("GPU details:")
    for gpu in gpu_available:
        print(gpu)
else:
    print("No GPU available.")

GPU is available.
GPU details:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [7]:
!date

Wed Oct 22 04:56:08 AM UTC 2025


In [8]:
!ls

Control			     gmx_MMPBSA.log  lupeol
final_verification_check.sh  hedragenin      maslinic_acid


!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&

To source the gmx command

In [9]:
%cd "/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol"

/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol


In [10]:
!ls

 ANTECHAMBER_AC.AC		 md_0_25.edr		 npt_prev.cpt
 ANTECHAMBER_AC.AC0		 md_0_25.gro		 npt.tpr
 ANTECHAMBER_AM1BCC.AC		 md_0_25.log		'#npt.tpr.1#'
 ANTECHAMBER_AM1BCC_PRE.AC	 md_0_25_prev.cpt	 npt.xtc
 ANTECHAMBER_BOND_TYPE.AC	 md_0_25.tpr		'#npt.xtc.1#'
 ANTECHAMBER_BOND_TYPE.AC0	 md_0_25.xtc		 nvt.cpt
 ATOMTYPE.INF			 md_25_50.cpt		 nvt.edr
 box.gro			 md_25_50.part0002.edr	'#nvt.edr.1#'
 cgenff_charmm2gmx_py3_nx1.py	 md_25_50.part0002.gro	 nvt.gro
 complex_1h6u_lup_70626379.pdb	 md_25_50.part0002.log	'#nvt.gro.1#'
 complex.gro			 md_25_50.part0002.xtc	 nvt.log
 em.edr				 md_25_50_prev.cpt	'#nvt.log.1#'
 em.gro				 md_25_50.tpr		 nvt.mdp
 em.log				 md_50_75.cpt		 nvt_prev.cpt
 em.mdp				 md_50_75.part0003.edr	 nvt.tpr
 em.tpr				 md_50_75.part0003.gro	'#nvt.tpr.1#'
 em.trr				 md_50_75.part0003.log	 nvt.xtc
'Gromacs Codes'			 md_50_75.part0003.xtc	'#nvt.xtc.1#'
 index.ndx			 md_50_75_prev.cpt	 posre.itp
'#index.ndx.1#'			 md_50_75.tpr		 protein.gro
 interaction_analysis.py	 md.mdp			 

In [ ]:
!perl sort_mol2_bonds.pl lig.mol2 lig.mol2

Found 57 atoms in the molecule, with 60 bonds.


In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx pdb2gmx -f protein.pdb -o protein.gro -ignh

                     :-) GROMACS - gmx pdb2gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx pdb2gmx -f protein.pdb -o protein.gro -ignh

Note that more recent versions of the CHARMM force field may be downloaded from
http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs.

Select the Force Field:

From '/content/drive/MyDrive/Colab_Software/GROMACS/share/gromacs/top':

 1: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 2: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 3: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 4: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 5: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006)

 6

In [ ]:
!pip install acpype

  Using cached acpype-2023.10.27-py3-none-any.whl.metadata (12 kB)
Using cached acpype-2023.10.27-py3-none-any.whl (28.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 98.1 MB/s eta 0:00:00


In [ ]:
conda_env_name

'gmxmmpbsa'

In [ ]:
print("--- 1. Running Antechamber ---")
!conda run -n {conda_env_name} antechamber -i lig.mol2 -fi mol2 -o lig_fixed.mol2 -fo mol2 -c bcc -s 2

--- 1. Running Antechamber ---

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for mol2 File --
   Status: pass
Info: Finished reading file (lig.mol2); atoms read (57), bonds read (60).
Info: Determining atomic numbers from atomic symbols which are case sensitive.
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Running: /usr/local/envs/gmxmmpbsa/bin/bondtype -j full -i ANTECHAMBER_BOND_TYPE.AC0 -o ANTECHAMBER_BOND_TYPE.AC -f ac

Running: /usr/local/envs/gmxmmpbsa/bin/atomtype -i ANTECHAMBER_AC.AC0 -

In [ ]:
print("\n--- 2. Running Parmchk2 ---")
!conda run -n {conda_env_name} parmchk2 -i lig_fixed.mol2 -f mol2 -o lig.frcmod -s gaff


--- 2. Running Parmchk2 ---


In [ ]:
# Ensure the Conda shell is initialized for this Python session
!source /usr/local/etc/profile.d/conda.sh

# 4. Generate GROMACS Topology with acpype
print("\nGenerating GROMACS Topology with acpype...")
!conda run -n gmxmmpbsa acpype -i lig_fixed.mol2 -b lig -c user -o gmx


Generating GROMACS Topology with acpype...
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2023.10.27 (c) 2025 AWSdS |
==> Reading user's charges from mol2 file...
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
==> * Tleap OK *
==> Removing temporary files...
==> Using OpenBabel v.3.1.0

==> Writing GROMACS files

==> Disambiguating lower and uppercase atomtypes in GMX top file, even if identical.

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig.pkl
==> Removing temporary files...
Total time of execution: 1s

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in ReadMolecule
  This Mol2 file is non-standard. Problem with molecule: complex_1h6u_lup_70626379.pdb Cannot interpret atom types correctly, instead attempting to interpret atom type: c3 as elements instead.
*** Open Babel Warning  in Translate

In [ ]:
import os
import shutil

# Define source and destination paths
source_itp = "lig.acpype/lig_GMX.itp"
source_gro = "lig.acpype/lig_GMX.gro"
destination_dir = "." # Current directory

# Copy ligand files
print(f"Copying {source_itp} to {destination_dir}")
shutil.copy(source_itp, destination_dir)

print(f"Copying {source_gro} to {destination_dir}")
shutil.copy(source_gro, destination_dir)

print("Ligand files copied successfully.")

Copying lig.acpype/lig_GMX.itp to .
Copying lig.acpype/lig_GMX.gro to .
Ligand files copied successfully.


In [ ]:
# Edit topol.top
topol_file = "topol.top"

# Read the original topol.top content
with open(topol_file, "r") as f:
    lines = f.readlines()

# Find the line after which to include the ligand topology
# This assumes the forcefield include is present and in the format like #include "amber99sb-ildn.ff/forcefield.itp"
insert_index = -1
for i, line in enumerate(lines):
    if line.strip().startswith("#include") and "forcefield.itp" in line:
        insert_index = i + 1
        break

if insert_index != -1:
    # Insert the ligand topology include and molecule definition
    ligand_include = '; Include ligand topology\n#include "lig_GMX.itp"\n'
    ligand_molecule = '\n[ molecules ]\n; Compound        #mols\nProtein_chain_A     1\nlig                 1\n'

    # Remove existing [ molecules ] section if it exists, to avoid duplicates
    molecules_section_start = -1
    for i, line in enumerate(lines):
        if line.strip() == '[ molecules ]':
            molecules_section_start = i
            break

    if molecules_section_start != -1:
        # Find the end of the molecules section (or end of file)
        molecules_section_end = len(lines)
        for i in range(molecules_section_start + 1, len(lines)):
            if line.strip().startswith('[') and not line.strip() == '[ molecules ]':
                 molecules_section_end = i
                 break
        lines = lines[:molecules_section_start] + lines[molecules_section_end:]


    # Insert the ligand include statement after the forcefield include
    lines.insert(insert_index, ligand_include)

    # Add the new molecules section at the end
    lines.extend(ligand_molecule.splitlines(keepends=True))


    # Write the modified content back to topol.top
    with open(topol_file, "w") as f:
        f.writelines(lines)

    print(f"Successfully modified {topol_file}")
else:
    print(f"Could not find the forcefield include in {topol_file}. Please manually edit the file.")

Successfully modified topol.top


In [ ]:
def combine_gro_files(protein_gro, ligand_gro, output_gro):
    """Combines protein and ligand .gro files."""

    with open(protein_gro, 'r') as f_prot:
        prot_lines = f_prot.readlines()

    with open(ligand_gro, 'r') as f_lig:
        lig_lines = f_lig.readlines()

    # Extract coordinate lines (skip header and box vector line)
    # Assuming standard .gro format: Title, num_atoms, coordinates, box_vectors
    prot_coords = prot_lines[2:-1]
    lig_coords = lig_lines[2:-1]

    # Get number of atoms
    num_prot_atoms = int(prot_lines[1].strip())
    num_lig_atoms = int(lig_lines[1].strip())
    total_atoms = num_prot_atoms + num_lig_atoms

    # Get box vectors from protein file (assuming they are the same)
    box_vectors = prot_lines[-1]

    # Write combined .gro file
    with open(output_gro, 'w') as f_out:
        f_out.write(prot_lines[0]) # Write title line
        f_out.write(f"{total_atoms}\n") # Write updated atom count
        f_out.writelines(prot_coords) # Write protein coordinates
        f_out.writelines(lig_coords) # Write ligand coordinates
        f_out.write(box_vectors) # Write box vectors

    print(f"Successfully combined {protein_gro} and {ligand_gro} into {output_gro}")

# Define input and output filenames
protein_gro_file = "protein.gro"
ligand_gro_file = "lig_GMX.gro"
complex_gro_file = "complex.gro"

# Run the function to combine files
combine_gro_files(protein_gro_file, ligand_gro_file, complex_gro_file)

Successfully combined protein.gro and lig_GMX.gro into complex.gro


In [ ]:
%%bash

# Define paths (optional, but good practice)
GROMACS_INSTALL_DIR="/content/drive/MyDrive/Colab_Software/GROMACS"
ENV_SCRIPT="${GROMACS_INSTALL_DIR}/gromacs_env.sh"

# Source the GROMACS environment script and run the command
source ${ENV_SCRIPT} && gmx editconf -f complex.gro -o box.gro -c -d 1.0 -bt triclinic

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 4702 atoms
Volume: 192.251 nm^3, corresponds to roughly 86500 electrons
No velocities found
    system size :  4.209  5.426  8.418 (nm)
    center      : -0.071  0.335  6.112 (nm)
    box vectors :  4.209  5.427  8.417 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 192.25               (nm^3)
    shift       :  3.176  3.378 -0.903 (nm)
new center      :  3.105  3.713  5.209 (nm)
new box vectors :  6.209  7.426 10.418 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 480.35               (nm^3)


                     :-) GROMACS - gmx editconf, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx editconf -f complex.gro -o box.gro -c -d 1.0 -bt triclinic


GROMACS reminds you: "It's Time to Move On" (F. Black)



In [ ]:
%%bash

# Define paths (optional, but good practice)
GROMACS_INSTALL_DIR="/content/drive/MyDrive/Colab_Software/GROMACS"
ENV_SCRIPT="${GROMACS_INSTALL_DIR}/gromacs_env.sh"

# Source the GROMACS environment script and run the command
source ${ENV_SCRIPT} && gmx solvate -cp box.gro -cs spc216.gro -p topol.top -o solvated.gro


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. (1964)
DOI: 10.1021/j100785a001
-------- -------- --- Thank You --- -------- --------

Adding line for 14341 solvent molecules with resname (SOL) to topology file (topol.top)


                     :-) GROMACS - gmx solvate, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx solvate -cp box.gro -cs spc216.gro -p topol.top -o solvated.gro

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 4x4x6 boxes
Solvent box contains 51948 atoms in 17316 residues
Removed 4599 solvent atoms due to solvent-solvent overlap
Removed 4326 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
    SOL (   3 atoms): 14341 residues
Generated solvent containing 43023 atoms in 14341 residues
Writing generated configuration to solvated.gro

Output configuration contains 47725 atoms in 14650 residues
Volume                 :     480.

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 1

                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 1


NOTE 1 [file ions.mdp]:
  For a correct single-point energy evaluation with nsteps = 0, use
  continuation = yes to avoid constraining the input coordinates.

Setting the LD random seed to 516865813

Generated 2775 of the 2775 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2775 of the 2775 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 3 bonded neighbours molecule type 'lig'

Excluding 2 bonded neighbours molecule type 'SOL'

NOTE 2 [file topol.top, line 44616]:
  System has non-zero total charge: -8.000000
  Total charge should 

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx genion -s ions.tpr -o ions.gro -p topol.top -pname NA -nname CL -neutral

                      :-) GROMACS - gmx genion, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx genion -s ions.tpr -o ions.gro -p topol.top -pname NA -nname CL -neutral

Reading file ions.tpr, VERSION 2025.3 (single precision)
Reading file ions.tpr, VERSION 2025.3 (single precision)
Will try to add 8 NA ions and 0 CL ions.
Select a continuous group of solvent molecules
Group     0 (         System) has 47725 elements
Group     1 (        Protein) has  4645 elements
Group     2 (      Protein-H) has  2296 elements
Group     3 (        C-alpha) has   308 elements
Group     4 (       Backbone) has   924 elements
Group     5 (      MainChain) has  1233 elements
Group     6 (   MainChain+Cb) has  1525 elements
Group     7 (    MainChain+H) has  1524 elements
Group     8 (      SideChain) has

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f min.mdp -c ions.gro -p topol.top -o em.tpr

                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx grompp -f min.mdp -c ions.gro -p topol.top -o em.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file min.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -28573729

Generated 2775 of the 2775 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2775 of the 2775 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 3 bonded neighbours molecule type 'lig'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded ne

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -v -deffnm em

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx mdrun -v -deffnm em

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file em.tpr, VERSION 2025.3 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 12 OpenMP threads 


Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+03
   Number of steps    =        50000
Step=    0, Dmax= 1.0e-02 nm, Epot= -2.85626e+05 Fmax= 1.72094e+05, atom= 1311
Step=    1, Dmax= 1.0e-02 nm, Epot= -3.44191e+05 Fmax= 6.24447e+04, atom= 2214
Step=    2, Dmax= 1.2e-02 nm, Epo

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx make_ndx -f em.gro -o index.ndx

                     :-) GROMACS - gmx make_ndx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx make_ndx -f em.gro -o index.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:   308    Protein residues
There are:     1      Other residues
There are: 14333      Water residues
There are:     8        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 47709 atoms
  1 Protein             :  4645 atoms
  2 Protein-H           :  2296 atoms
  3 C-alpha             :   308 atoms
  4 Backbone            :   924 atoms
  5 MainChain           :  1233 atoms
  6 MainChain+Cb        :  1525 atoms
  7 MainChain+H         :  1524 atoms
  8 SideChain         

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr -n index.ndx


                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr -n index.ndx

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Setting the LD random seed to -69370945

Generated 2775 of the 2775 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2775 of the 2775 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'lig'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into const

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -v -deffnm nvt

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx mdrun -v -deffnm nvt


Back Off! I just backed up nvt.log to ./#nvt.log.1#
Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file nvt.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 12

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -p topol.top -o npt.tpr -n index.ndx -maxwarn 1


                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -p topol.top -o npt.tpr -n index.ndx -maxwarn 1

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

WARNING 1 [file npt.mdp]:
  The Berendsen barostat does not generate any strictly correct ensemble,
  and should not be used for new production simulations (in our opinion).
  We recommend using the C-rescale barostat instead.

Setting the LD random seed to -1610943171

Generated 2775 of the 2775 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2775 of the 2775 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excludin

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -v -deffnm npt

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx mdrun -v -deffnm npt


Back Off! I just backed up npt.log to ./#npt.log.1#
Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file npt.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 12

In [ ]:
!ls

 ANTECHAMBER_AC.AC		 lig.acpype	 '#nvt.edr.1#'
 ANTECHAMBER_AC.AC0		 lig_fixed.mol2   nvt.gro
 ANTECHAMBER_AM1BCC.AC		 lig.frcmod	 '#nvt.gro.1#'
 ANTECHAMBER_AM1BCC_PRE.AC	 lig_GMX.gro	  nvt.log
 ANTECHAMBER_BOND_TYPE.AC	 lig_GMX.itp	 '#nvt.log.1#'
 ANTECHAMBER_BOND_TYPE.AC0	 lig.mol2	  nvt.mdp
 ATOMTYPE.INF			 md.mdp		  nvt_prev.cpt
 box.gro			 mdout.mdp	  nvt.tpr
 cgenff_charmm2gmx_py3_nx1.py	 min.mdp	 '#nvt.tpr.1#'
 complex_1h6u_lup_70626379.pdb	 npt.cpt	  nvt.xtc
 complex.gro			 npt.edr	 '#nvt.xtc.1#'
 em.edr				'#npt.edr.1#'	  posre.itp
 em.gro				 npt.gro	  protein.gro
 em.log				'#npt.gro.1#'	  protein.pdb
 em.mdp				 npt.log	  solvated.gro
 em.tpr				'#npt.log.1#'	  sort_mol2_bonds.pl
 em.trr				 npt.mdp	  sqm.in
'Gromacs Codes'			 npt_prev.cpt	  sqm.out
 index.ndx			 npt.tpr	  sqm.pdb
'#index.ndx.1#'			'#npt.tpr.1#'	  topol.top
 interaction_analysis.py	 npt.xtc	 '#topol.top.1#'
 ions.gro			'#npt.xtc.1#'	 '#topol.top.2#'
 ions.mdp			 nvt.cpt
 ions.tpr			 nvt.edr




> **!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&**



In [ ]:
import os

# Get the path to the md.mdp file
mdp_file = "md.mdp"

if os.path.exists(mdp_file):
    print("--- Fixing md.mdp file ---")

    # This command finds the line starting with 'tc-grps' and replaces it
    # to use the two default groups: 'Protein' and 'non-Protein'.
    !sed -i 's/^tc-grps.*/tc-grps = Protein non-Protein/' {mdp_file}

    # This command finds the 'tau_t' line (coupling time)
    # and ensures it provides two values, one for each group.
    !sed -i 's/^tau_t.*/tau_t = 0.1 0.1/' {mdp_file}

    print("md.mdp has been fixed.")
else:
    print(f"Error: {mdp_file} not found.")

--- Fixing md.mdp file ---
md.mdp has been fixed.


# --- Block 1 (0-25 ns) ---

In [ ]:
# --- Block 1 (0-25 ns) ---
print("--- 4a. Running Block 1 (0-25 ns) ---")
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f md.mdp -c npt.gro -p topol.top -o md_0_25.tpr -maxwarn 1


--- 4a. Running Block 1 (0-25 ns) ---
                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx grompp -f md.mdp -c npt.gro -p topol.top -o md_0_25.tpr -maxwarn 1

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Setting the LD random seed to 1360522027

Generated 2775 of the 2775 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2775 of the 2775 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'lig'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type '

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_0_25.tpr -deffnm md_0_25 -v -cpt 0.5

# --- Block 2 (25-50 ns) ---

In [ ]:
print("\n--- 4b. Running Block 2 (25-50 ns) ---")
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx convert-tpr -s md_0_25.tpr -o md_25_50.tpr -extend 25000
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_25_50.tpr -deffnm md_25_50 -v -cpt 0.5 -cpi md_0_25.cpt -noappend


--- 4b. Running Block 2 (25-50 ns) ---
                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_25_50.tpr -extend 25000

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 25000 ps

Output file:
  Run start step                                     0     
  Run start time                                     0

# --- Block 3 (50-75 ns) ---

In [ ]:
print("\n--- 4c. Running Block 3 (50-75 ns) ---")
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&  gmx convert-tpr -s md_0_25.tpr -o md_50_75.tpr -extend 50000
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&  gmx mdrun -s md_50_75.tpr -deffnm md_50_75 -v -cpt 0.5 -cpi md_25_50.cpt -noappend


--- 4c. Running Block 3 (50-75 ns) ---
                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_50_75.tpr -extend 50000

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 50000 ps

Output file:
  Run start step                                     0     
  Run start time                                     0

# --- Block 4 (75-100 ns) ---

In [11]:
print("\n--- 4d. Running Block 4 (75-100 ns) ---")
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx convert-tpr -s md_0_25.tpr -o md_75_100.tpr -extend 75000
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_75_100.tpr -deffnm md_75_100 -v -cpt 0.5 -cpi md_50_75.cpt -noappend


--- 4d. Running Block 4 (75-100 ns) ---
                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_75_100.tpr -extend 75000

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 75000 ps

Output file:
  Run start step                                     0     
  Run start time                                    

# --- Block 5 (100-125 ns) ---

In [12]:
print("\n--- 4e. Running Block 5 (100-125 ns) ---")
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&  gmx convert-tpr -s md_0_25.tpr -o md_100_125.tpr -extend 100000
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&  gmx mdrun -s md_100_125.tpr -deffnm md_100_125 -v -cpt 0.5 -cpi md_75_100.cpt -noappend


--- 4e. Running Block 5 (100-125 ns) ---
                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/lupeol
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_100_125.tpr -extend 100000

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 100000 ps

Output file:
  Run start step                                     0     
  Run start time                                

In [13]:
print("MD run has finished.")
print("Shutting down Colab runtime to save points...")

from google.colab import runtime
runtime.unassign()

MD run has finished.
Shutting down Colab runtime to save points...
